In [ ]:
# architecture 1 taken from paper
# maybe later add more architectures
hidden_layer_sizes = [20, 10, 5]
dropout_rates_train = [0, 0, 0, 0]
activation_functions = ["relu", "relu", "relu", "none"]

In [ ]:
# propensity scores taken from paper

hidden_layer_sizes_treatment = [50, 30]
activation_functions_treatment = ["relu", "relu", "none"]
dropout_rates_train_treatment = [0, 0, 0]
dropout_rates_test_treatment = [0 for i in dropout_rates_train_treatment]

In [ ]:
# hyperparameters

train_proportion = 0.9
max_nepochs = 5000
max_epochs_without_change = 30
optimizer = "Adam"
learning_rate = 0.009
batch_size = 128
batch_size_t = None

In [ ]:
# Regularization parameters
alpha = 0.0
r = 0.2

In [ ]:
import pandas as pd

# Load the real data
data = pd.read_csv(
    "bld/difference_in_difference_with_deep_learning/data/cleaned_data.csv",
)  # Change the filename to your actual data file

# Assuming 'wage_year' is the variable of interest
X = data[["Individual", "Age", "WagePartner"]]  # Features
T_real = data["interaction"]  # Treatment assignment
Y = data["wage_year"]  # Outcome

In [ ]:
df = data

In [ ]:
# one hot encoded

In [ ]:
# Convert time variable to numerical values
time_mapping = {"t-2": -2, "t-1": -1, "t+1": 1, "t+2": 2, "t+3": 3}
df["time"] = df["time"].map(time_mapping)

In [ ]:
df

from fakedata get some values
- mu0_real
- tau_real
- T_real
- seed
- prob_of_T
- tau_true_mean

which functions do I need:
- _sum_polynomial_X_times_weights
- _create_TE_coefs
- _calculate_true_tau_mean
- _create_propensity_scores
- create_fake_data, just to create the datapoints

In [ ]:
from itertools import combinations_with_replacement

import numpy as np

In [ ]:
feature = df.drop("wage_year", axis=1).values
Target = df["wage_year"].values.reshape(-1, 1)

In [ ]:
def evaluate_quadratic_polynomial(features, weights):
    """Evaluate the non-linear part of a quadratic polynomial in
    consumer characteristics, features, with prescribed weights.

    Inputs:
    -------
        features: ndarray, shape = (N, n_features)
            Input features.
        weights: ndarray, shape = (num_additional_poly_terms, )
            Weights corresponding to quadratic terms.
    Outputs:
    -------
        sum_x: ndarray, shape = (N, 1)
            Non-linear part of the quadratic polynomial evaluated
            for each sample.
    """
    n_features = features.shape[1]
    my_polynomial_indices = combinations_with_replacement(list(range(n_features)), 2)
    i = 0
    sum_x = 0
    for p in my_polynomial_indices:
        sum_x = sum_x + weights[i % len(weights)] * np.multiply(
            features[:, p[0]],
            features[:, p[1]],
        )
        i += 1
    return sum_x.reshape(-1, 1)


num_additional_poly_terms = 10

# Generate random weights
weights = np.random.uniform(low=-1.0, high=1.0, size=num_additional_poly_terms)

# Call the function for X_train
sum_x = evaluate_quadratic_polynomial(feature, weights)

In [ ]:
from scipy.special import comb

In [ ]:
def _create_TE_coefs(features, treatment_interaction, model):
    """Create treatment effect coefficients.

    Inputs:
    -------
        features: ndarray, shape = (N, n_features)
            Input features.
        treatment_interaction: ndarray, shape = (N, 1)
            Interaction term from the data.
        model: {'simple', 'quadratic'}
            If 'simple' coefficients a and b in the artificial
            dataset depend linearly on consumer characteristics.
            Otherwise, the dependence is quadratic.
    Outputs:
    -------
        bias_tau: float
            Constant term in equation for tau.
        alpha_tau: ndarray, shape = [n_features, 1]
            Linear coefficients in equation for tau.
        beta_tau: ndarray, shape = [count]
            Quadratic coefficients in equation for tau.
            Count is the number of the second degree terms in a
            quadratic polynomial where the number of variables is
            equal to the number of consumer characteristics.
    """
    # Concatenate features and treatment_interaction
    features_with_interaction = np.concatenate(
        (features, treatment_interaction),
        axis=1,
    )

    # Calculating tau
    alpha_tau = np.random.uniform(
        low=0.1,
        high=0.22,
        size=[features_with_interaction.shape[1], 1],
    )
    bias_tau = -0.05
    tau = np.dot(features_with_interaction, alpha_tau) + bias_tau

    if model == "quadratic":
        num_additional_poly_terms = comb(
            features_with_interaction.shape[1],
            2,
            True,
            True,
        )
        beta_tau = np.random.uniform(
            low=-0.05,
            high=0.06,
            size=num_additional_poly_terms,
        )
        sum_x = evaluate_quadratic_polynomial(features_with_interaction, beta_tau)
        tau += sum_x
    else:
        beta_tau = None

    return alpha_tau, bias_tau, beta_tau

In [ ]:
X = df[["time", "wage_year", "FQ", "Reform", "Age", "WagePartner"]].values
treatment_interaction = df["interaction"].values.reshape(
    -1,
    1,
)  # Assuming 'interaction' is the treatment interaction term

# Call the function for X
alpha_tau, bias_tau, beta_tau = _create_TE_coefs(X, treatment_interaction, "quadratic")

In [ ]:
def _calculate_true_tau_mean(
    alpha_tau,
    bias_tau,
    beta_tau,
    treatment_interaction,
    model,
):
    """Calculate true average treatment effect.

    Inputs:
    -------
        bias_tau: float
            Constant term in equation for tau.
        alpha_tau: ndarray, shape = [nconsumer_characteristics, 1]
            Linear coefficients in equation for tau.
        beta_tau: ndarray, shape = [count]
            Quadratic coefficients in equation for tau.
            Count is the number of the second degree terms in a
            quadratic polynomial where the number of variables is
            equal to the number of consumer characteristics.
        treatment_interaction: ndarray, shape = (N, 1)
            Interaction term representing the treatment.
        model: {'simple', 'quadratic'}
            If 'simple' coefficients a and b in the artificial
            dataset depend linearly on consumer characteristics.
            Otherwise, the dependence is quadratic.
    """
    X = treatment_interaction

    tau_true_mean = np.sum(X * alpha_tau) + bias_tau

    if model == "quadratic":
        X_poly = 0.25 * np.ones(len(beta_tau))
        s = 0
        for i in range(treatment_interaction.shape[1]):
            X_poly[s] = 1 / 3.0
            s = s + treatment_interaction.shape[1] - i

        tau_true_mean = tau_true_mean + np.sum(X_poly * beta_tau)

    return tau_true_mean


# Assuming you have already computed the alpha_tau, bias_tau, beta_tau
tau_true_mean = _calculate_true_tau_mean(
    alpha_tau,
    bias_tau,
    beta_tau,
    treatment_interaction,
    "quadratic",
)

# additional

In [ ]:
# needed?
hello = 1
# the question is whether train and test split are in the part beforehand relevant

df = df.sort_values(by="time").reset_index(drop=True)

# Determine the index to split the data into training and testing sets
split_index = int(0.8 * len(df))  # 80% training, 20% testing

# Split the DataFrame into training and testing sets
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]

# Separate features and target for training and testing sets
X_train = train_df.drop("wage_year", axis=1).values
Y_train = train_df["wage_year"].values.reshape(-1, 1)
X_test = test_df.drop("wage_year", axis=1).values
Y_test = test_df["wage_year"].values.reshape(-1, 1)